# Concordia <> Mastodon Example

# Introduction

**Purpose of the Simulation**

This notebook demonstrates a generative agent simulation of a Mastodon instance using the Concordia framework. 
The simulation models user behavior on a local Mastodon server, showcasing how AI agents interact in a 
decentralized social media environment.

## Setup and imports

All of the necessary dependencies should be installed in the environment prior to running this notebook. If not, install them using poetry:

```bash
poetry install
```

This will include installing the Concordia package from a fork that includes a branch with a fix for a bug in the calendar/phone example.

Alternatively, you may install the library in editable mode so that we can make changes to the code if necessary:

```bash
git clone https://github.com/austinmw/concordia.git
cd concordia
git checkout fix-calendar-example
pip install -e . --config-settings editable_mode=compat
```

In [ ]:
%load_ext autoreload
%autoreload 2

import concurrent.futures
import datetime
import os
import random
import warnings

with warnings.catch_warnings():
    warnings.filterwarnings("ignore")
    import sentence_transformers

from concordia import components as generic_components
from concordia.agents import basic_agent
from concordia.associative_memory import (
    associative_memory,
    blank_memories,
    formative_memories,
    importance_function,
)
from concordia.clocks import game_clock
from concordia.components import agent as components
from concordia.components import game_master as gm_components
from concordia.environment import game_master
from concordia.language_model import amazon_bedrock_model, gpt_model
from concordia.metrics import (  # noqa: F401
    common_sense_morality,
    goal_achievement,
    opinion_of_others,
)
from concordia.utils import html as html_lib
from concordia.utils import measurements as measurements_lib
from IPython import display

from mastodon_sim.concordia import apps, triggering
from mastodon_sim.mastodon_ops import check_env, get_public_timeline, print_timeline, reset_users
from mastodon_sim.mastodon_utils import get_users_from_env

## Check environment
Check for a `.env` file with the nessesary environment variables. You should see an output similar to this if successful:

```log
2024-07-17T11:38:17.213066-0400 INFO Successfully loaded .env file.
2024-07-17T11:38:17.214151-0400 INFO API_BASE_URL: https://social-sandbox.com
2024-07-17T11:38:17.214581-0400 INFO EMAIL_PREFIX: austinmw89
2024-07-17T11:38:17.215056-0400 INFO MASTODON_CLIENT_ID: N*****************************************E
2024-07-17T11:38:17.215477-0400 INFO MASTODON_CLIENT_SECRET: b*****************************************s
2024-07-17T11:38:17.215877-0400 INFO USER0005_PASSWORD: a******************************a
2024-07-17T11:38:17.216202-0400 INFO USER0002_PASSWORD: 8******************************b
2024-07-17T11:38:17.216655-0400 INFO USER0001_PASSWORD: 9******************************5
2024-07-17T11:38:17.217020-0400 INFO USER0004_PASSWORD: 7******************************e
2024-07-17T11:38:17.217323-0400 INFO USER0003_PASSWORD: f******************************6
```

In [ ]:
check_env()

## Clear Mastodon server
Deletes from https://social-sandbox.com all posts, follows, likes, etc. from all users in the .env file.

In [ ]:
# Get all users from .env file
users = get_users_from_env()

# Reset all users
reset_users(users, skip_confirm=True, parallel=True)

# Check that the public timeline is now empty
assert not len(get_public_timeline(limit=None))

## Set LLM and embedding model

In [ ]:
# Language Model setup
GPT_API_KEY = os.getenv("OPENAI_API_KEY")
GPT_MODEL_NAME = "gpt-4o"

if not GPT_API_KEY and "sonnet" not in GPT_MODEL_NAME.lower():
  raise ValueError("GPT_API_KEY is required.")

if "sonnet" in GPT_MODEL_NAME.lower():
  model = amazon_bedrock_model(
    # TODO: replace with Sonnet V3.5 (requested access)
    model_id="anthropic.claude-3-sonnet-20240229-v1:0"
  )
else:
  model = gpt_model.GptLanguageModel(
      api_key=GPT_API_KEY, model_name=GPT_MODEL_NAME
  )

In [ ]:
# Setup sentence encoder
st_model = sentence_transformers.SentenceTransformer(
    "sentence-transformers/all-mpnet-base-v2"
)

embedder = lambda x: st_model.encode(x, show_progress_bar=False)  # noqa: E731

## Configuring the generic knowledge of the players and the game master (GM)

In [ ]:
INSTANCE_NAME = "riverbend.social"

shared_memories = [
    # Backstory
    f"You are a user on {INSTANCE_NAME}, a Mastodon instance created for the residents of Riverbend.",
    "Riverbend is a small, picturesque town known for its strong community spirit and love for nature.",
    f"{INSTANCE_NAME} was founded by the town's tech-savvy mayor to encourage local online engagement.",
    "The instance has been active for about a year and has become integral to Riverbend's community life.",
    "Many local businesses, community groups, and residents are active on the platform.",
    # Riverbend-specific details
    "Riverbend has a beautiful river running through it, which is central to the town's identity and economy.",
    "The town is facing some environmental challenges due to climate change, which is a frequent topic of discussion.",
    f"There's a popular hashtag #RiverbendDaily on {INSTANCE_NAME} where residents share local news and events.",
    f"{INSTANCE_NAME} has monthly online town halls where residents can discuss local issues.",
    "The mayor regularly posts updates about town developments and initiatives on the platform.",
    # Basic Mastodon knowledge
    "Mastodon is a decentralized social network, different from centralized platforms like Twitter or Facebook.",
    "To share content on Mastodon, you write a 'toot' (equivalent to a tweet or post).",
    "Toots can be up to 500 characters long, allowing for more detailed expressions than some other platforms.",
    # Timeline and Visibility
    "Your home timeline shows toots from people you follow and boosted (reblogged) content.",
    "The local timeline displays public toots from users on your instance.",
    "The timeline is not algorithmically curated, and toots appear in reverse chronological order.",
    # Interactions
    "You can reply to toots, creating threaded conversations.",
    "Favorite (like) toots to show appreciation or save them for later.",
    "Boost (reblog) toots to share them with your followers.",
    "You can mention other users in your toots using their @username.",
    # Following and Followers
    "Follow other users to see their public and unlisted toots in your home timeline.",
    "You can unfollow users if you no longer wish to see their content.",
    "Users don't need to follow each other mutually; it can be one-sided.",
    # Discovery and Features
    "Use hashtags to categorize your toots and make them more discoverable.",
    "The explore page shows trending hashtags and featured toots.",
    "Your profile can be customized with a display name and bio.",
    # Privacy and Safety
    "You can block users to prevent them from seeing your content or interacting with you.",
    "Unblocking a user reverses the effects of blocking.",
    f"{INSTANCE_NAME} has a strict policy against misinformation and encourages fact-checking.",
    # Community Guidelines
    f"As a user of {INSTANCE_NAME}, you're expected to engage respectfully and contribute positively to the community.",
    "Discussions about local environmental issues, community events, and town developments are encouraged.",
    "While debating is welcome, personal attacks and harassment are not tolerated on the platform.",
    # Realistic User Behavior
    "Despite community guidelines, users sometimes engage in heated arguments, especially on controversial topics.",
    "Some users tend to share unverified information or sensationalized news, particularly during local emergencies or elections.",
    "Cliques often form among users with similar interests or viewpoints, sometimes leading to echo chambers.",
    "While many users are supportive, instances of trolling or targeted harassment occasionally occur.",
    "Some users frequently post about personal matters or overshare details about their daily lives.",
    "Local businesses sometimes use the platform for excessive self-promotion, which can annoy other users.",
    "During major local events or crises, the instance can become flooded with repetitive information and speculation.",
    "A few users have multiple accounts, sometimes using alternate personas to engage differently with the community.",
    "Despite the longer character limit, many users still prefer short, quippy posts over more thoughtful, lengthy discussions.",
    "Some users tend to boost and interact mainly with popular or influential accounts, seeking to increase their own visibility.",
]

In [ ]:
# The generic context will be used for the NPC context. It reflects general
# knowledge and is possessed by all characters.
shared_context = model.sample_text(
    "Summarize the following passage in a concise and insightful fashion:\n"
    + "\n".join(shared_memories)
    + "\n"
    + "Summary:",
    max_tokens=512,
)
print(shared_context)
importance_model = importance_function.ConstantImportanceModel()
importance_model_gm = importance_function.ConstantImportanceModel()

In [ ]:
# Make the clock
SETUP_TIME = datetime.datetime(year=2024, month=9, day=1, hour=8)  # noqa: DTZ001

START_TIME = datetime.datetime(year=2024, month=10, day=1, hour=8)  # noqa: DTZ001

clock = game_clock.MultiIntervalClock(
    start=SETUP_TIME,
    step_sizes=[datetime.timedelta(minutes=15), datetime.timedelta(seconds=10)]
)

## Functions to build the players

In [ ]:
blank_memory_factory = blank_memories.MemoryFactory(
    model=model,
    embedder=embedder,
    importance=importance_model.importance,
    clock_now=clock.now,
)

formative_memory_factory = formative_memories.FormativeMemoryFactory(
    model=model,
    shared_memories=shared_memories,
    blank_memory_factory_call=blank_memory_factory.make_blank_memory,
)

# All players get the same `measurements` object.
measurements = measurements_lib.Measurements()

In [ ]:
def build_agent(agent_config, measurements: measurements_lib.Measurements) -> tuple:
    """Build an agent based on the given configuration."""
    mem = formative_memory_factory.make_memories(agent_config)

    time = generic_components.report_function.ReportFunction(
        name="Current time",
        function=clock.current_time_interval_str,
    )

    somatic_state = components.somatic_state.SomaticState(
        model, mem, agent_config.name, clock.now
    )
    identity = components.identity.SimIdentity(
        model=model,
        memory=mem,
        agent_name=agent_config.name,
        clock_now=clock.now,
    )
    goal_component = components.constant.ConstantComponent(state=agent_config.goal)
    plan = components.plan.SimPlan(
        model=model,
        memory=mem,
        agent_name=agent_config.name,
        clock_now=clock.now,
        components=[identity],
        goal=goal_component,
        verbose=False,
    )
    current_obs = components.observation.Observation(
        agent_name=agent_config.name,
        clock_now=clock.now,
        memory=mem,
        timeframe=clock.get_step_size(),
        component_name="current observations",
    )
    summary_obs = components.observation.ObservationSummary(
        agent_name=agent_config.name,
        model=model,
        clock_now=clock.now,
        memory=mem,
        timeframe_delta_from=datetime.timedelta(hours=4),
        timeframe_delta_until=datetime.timedelta(minutes=15),
        components=[identity],
        component_name="summary of observations",
    )
    # goal_metric = goal_achievement.GoalAchievementMetric(
    #     model=model,
    #     player_name=agent_config.name,
    #     player_goal=agent_config.goal,
    #     clock=clock,
    #     name="Goal Achievement",
    #     measurements=measurements,
    #     channel="goal_achievement",
    #     verbose=False,
    # )
    # morality_metric = common_sense_morality.CommonSenseMoralityMetric(
    #     model=model,
    #     player_name=agent_config.name,
    #     clock=clock,
    #     name="Morality",
    #     verbose=False,
    #     measurements=measurements,
    #     channel="common_sense_morality",
    # )

    agent = basic_agent.BasicAgent(
        model=model,
        agent_name=agent_config.name,
        clock=clock,
        verbose=True,
        components=[
            identity,
            plan,
            somatic_state,
            summary_obs,
            current_obs,
            time,
            # goal_metric,
            # morality_metric,
        ],
    )

    # reputation_metric = opinion_of_others.OpinionOfOthersMetric(
    #     model=model,
    #     player_name=agent_config.name,
    #     player_names=player_names,
    #     context_fn=agent.state,
    #     clock=clock,
    #     name="Opinion",
    #     verbose=False,
    #     measurements=measurements,
    #     channel="opinion_of_others",
    #     question="What is {opining_player}'s opinion of {of_player}?",
    # )
    # agent.add_component(reputation_metric)

    return agent, mem

## Configure and build the players

In [ ]:
NUM_PLAYERS = 2

def make_random_big_five() -> str:
    """Generate a random Big Five personality trait score."""
    return str({
        "extraversion": random.randint(1, 10),
        "neuroticism": random.randint(1, 10),
        "openness": random.randint(1, 10),
        "conscientiousness": random.randint(1, 10),
        "agreeableness": random.randint(1, 10),
    })


scenario_premise = [
    # (
    #     "Alice, Bob, Charlie and Dorothy are at the Sundrop Saloon. There "
    #     "is a snow storm and they have to wait it out inside."
    # )
    (
        "It's early October in Riverbend, and the town is buzzing with activity ."
        "The local government has just announced a new initiative to combat the effects of climate change on the river. "
         "Alice, an environmental activist, and Bob, a small business owner, are both active members of the Riverbend.social Mastodon instance. "
         "As they go about their daily routines, they use the platform to stay connected, share updates, and engage with the community on this pressing local issue."
    )
]

player_configs = [
    formative_memories.AgentConfig(
        name="Alice",
        gender="female",
        #goal="Have fun interacting on social media.",
        goal="Balance work responsibilities with community engagement on Riverbend.social, aiming to organize a local environmental cleanup event.",
        context=f"{shared_context}\nAlice grew up in Riverbend.",
        traits=make_random_big_five()
    ),
    formative_memories.AgentConfig(
        name="Bob",
        gender="male",
        #goal="Have fun interacting on social media.",
        goal="Promote my small business on Riverbend.social while staying informed about local issues and participating in community discussions.",
        context=f"{shared_context}\nBob grew up in Riverbend.",
        traits=make_random_big_five()
    ),
]

In [ ]:
# Build the agents
from functools import partial

player_configs = player_configs[:NUM_PLAYERS]
player_goals = {
    player_config.name: player_config.goal for player_config in player_configs}
players = []
memories = {}

build_agent_with_arg = partial(build_agent, measurements=measurements)

with concurrent.futures.ThreadPoolExecutor(max_workers=NUM_PLAYERS) as pool:
    for agent, mem in pool.map(build_agent_with_arg, player_configs):
        players.append(agent)
        memories[agent.name] = mem

## Build the GM

In [ ]:
game_master_memory = associative_memory.AssociativeMemory(
    embedder, importance_model_gm.importance, clock=clock.now)

In [ ]:
# Add some memories to the game master
for player in players:
  game_master_memory.add(f"{player.name} is at their private home.")

In [ ]:
# Create components and externalities for the game master
citizen_names = [player.name for player in players]
player_names = [player.name for player in players]

facts_on_village = components.constant.ConstantComponent(" ".join(shared_memories), "General knowledge of Riverbend")
player_status = gm_components.player_status.PlayerStatus(clock.now, model, game_master_memory, player_names)

relevant_events = gm_components.relevant_events.RelevantEvents(clock.now, model, game_master_memory)
time_display = gm_components.time_display.TimeDisplay(clock)

direct_effect_externality = gm_components.direct_effect.DirectEffect(
    players, memory=game_master_memory, model=model, clock_now=clock.now, verbose=False, components=[player_status]
)

In [ ]:
# Create apps and provide them to the phones, assigning 1 phone to each player
calendar_app = apps.ToyCalendar()
mastodon_app = apps.MastodonSocialNetworkApp()

phones = [
    apps.Phone("Alice", apps=[calendar_app, mastodon_app]),
    apps.Phone("Bob", apps=[calendar_app, mastodon_app])
]

In [ ]:
# Update username for each person (since this is differnt from their name)
user_mapping = {
    "Bob": "user0001",
    "Alice": "user0002",
}
mastodon_app.set_user_mapping(user_mapping)

In [ ]:
# Seed follower relationships (simple all-to-all, for now)
for follower in user_mapping:
    for followee in user_mapping:
        if follower != followee:
            mastodon_app.follow_user(follower, followee)

In [ ]:
# Update Mastodon display name for each person
for name in user_mapping:
    mastodon_app.update_profile(name, display_name=name, bio="")

In [ ]:
# Create component to trigger PhoneGameMaster
phone_triggering = triggering.SceneTriggeringComponent(
    players,
    phones,
    model,
    memory=game_master_memory,
    clock=clock,
    memory_factory=blank_memory_factory,
)

In [ ]:
# Create the game master object
env = game_master.GameMaster(
    model=model,
    memory=game_master_memory,
    clock=clock,
    players=players,
    components=[
        facts_on_village,
        player_status,
        direct_effect_externality,
        relevant_events,
        time_display,
        phone_triggering,
    ],
    randomise_initiative=True,
    player_observes_event=False,
    verbose=True,
)

## The RUN

In [ ]:
clock.set(START_TIME)

In [ ]:
for player in players:
  player.observe( f"{player.name} is at home, they have just woken up.")
  player.observe( f"{player.name} remembers they need to update their Mastodon bio!")

In [ ]:
# @title Expect about 2-3 minutes per step.
episode_length = 5
for _ in range(episode_length):
  env.step()

## Summary and analysis of the episode

In [ ]:
# @title Summarize the entire story
all_gm_memories = env._memory.retrieve_recent(k=10000, add_time=True)

detailed_story = "\n".join(all_gm_memories)
print("len(detailed_story): ", len(detailed_story))
# print(detailed_story)

episode_summary = model.sample_text(
    f"Sequence of events:\n{detailed_story}"
    "\nNarratively summarize the above temporally ordered "
    "sequence of events. Write it as a news report. Summary:\n",
     max_tokens=3500,
     terminators=(),
)
print(episode_summary)

In [ ]:
# Summarise the perspective of each player
player_logs = []
player_log_names = []
for player in players:
  name = player.name
  detailed_story = "\n".join(memories[player.name].retrieve_recent(
      k=1000, add_time=True))
  summary = ""
  summary = model.sample_text(
      f"Sequence of events that happened to {name}:\n{detailed_story}"
      "\nWrite a short story that summarises these events.\n",
       max_tokens=3500,
       terminators=()
)

  all_player_mem = memories[player.name].retrieve_recent(k=1000, add_time=True)
  all_player_mem = ["Summary:", summary, "Memories:", *all_player_mem]
  player_html = html_lib.PythonObjectToHTMLConverter(all_player_mem).convert()
  player_logs.append(player_html)
  player_log_names.append(f"{name}")

## Build and display HTML log of the experiment

In [ ]:
history_sources = [env, direct_effect_externality]
histories_html = [html_lib.PythonObjectToHTMLConverter(history.get_history()).convert() for history in history_sources]
histories_names = [history.name for history in history_sources]

In [ ]:
gm_mem_html = html_lib.PythonObjectToHTMLConverter(all_gm_memories).convert()

tabbed_html = html_lib.combine_html_pages(
    [*histories_html, gm_mem_html, *player_logs],
    [*histories_names, "GM", *player_log_names],
    summary=episode_summary,
    title="Mastodon experiment",
)

tabbed_html = html_lib.finalise_html(tabbed_html)

In [ ]:
display.HTML(tabbed_html)

## Interact with a specific player

In [ ]:
sim_to_interact = "Alice"
user_identity = "a close friend"
interaction_premise = f"{sim_to_interact} is talking to {user_identity}\n"

player_names = [player.name for player in players]
player_by_name = {player.name: player for player in players}
selected_player = player_by_name[sim_to_interact]
interrogation = interaction_premise

In [ ]:
utterance_from_user = (
    "Hey Alice, did you post anything on Mastodon today?"
)

interrogation += f"{user_identity}: {utterance_from_user}"
player_says = selected_player.say(interrogation)
interrogation += f"\n{sim_to_interact}: {player_says}\n"
print(interrogation)

## Check timeline

Finally, check the full public timeline for the Mastodon server.

You may also check this directly at https://social-sandbox.com (you'll need to log in as a user).

In [ ]:
timeline = get_public_timeline(limit=None)
print_timeline(timeline)